# Udacity A/B Testing Final Project 
*By Jinghong Jiangyang*
1. [Experiment Overview: Free Trial Screener](#overview) <br>
2. [Experiment Design](#design) <br>
2.1 [Metric Choice](#choice) <br>
2.2 [Measuring Variability](#variability) <br>
2.3 [Sizing](#sizing) <br>
2.3.1 [Choosing Number of Samples Given Power](#samplesize) <br>
2.3.2 [Choosing Duration vs. Exposure](#duration) <br>
3. [Experiment Analysis](#analysis) <br>
3.1 [Sanity Checks](#checks) <br>
3.2 [Result Analysis](#resultanalysis) <br>
3.2.1 [Effect Size Tests](#effectsize) <br>
3.2.2 [Sign Tests](#signtest) <br>
3.2.3 [Summary](#summary) <br>
3.3 [Recommendation](#recommendation) <br>
4. [Follow-Up Experiment: How to Reduce Early Cancellations](#followup) <br>

## 1. Experiment Overview: Free Trial Screener<a class="anchor" id="overview"></a>

At the time of this experiment, Udacity courses currently have two options on the
course overview page: "start free trial", and "access course materials". If the
student clicks "start free trial", they will be asked to enter their credit card
information, and then they will be enrolled in a free trial for the paid version of the
course. After 14 days, they will automatically be charged unless they cancel first.
If the student clicks "access course materials", they will be able to view the videos
and take the quizzes for free, but they will not receive coaching support or a
verified certificate, and they will not submit their final project for feedback.


In the experiment, Udacity tested a change where if the student clicked "start free
trial", they were asked how much time they had available to devote to the course.
If the student indicated 5 or more hours per week, they would be taken through
the checkout process as usual. If they indicated fewer than 5 hours per week, a
message would appear indicating that Udacity courses usually require a greater
time commitment for successful completion, and suggesting that the student might
like to access the course materials for free. At this point, the student would have
the option to continue enrolling in the free trial, or access the course materials for
free instead. This screenshot shows what the experiment looks like. 

<img src="files/picture/image.png" width="700"/> 

The hypothesis was that this might set clearer expectations for students upfront,
thus reducing the number of frustrated students who left the free trial because
they didn't have enough time—without significantly reducing the number of
students to continue past the free trial and eventually complete the course. If this
hypothesis held true, Udacity could improve the overall student experience and
improve coaches' capacity to support students who are likely to complete the
course.


The unit of diversion is a cookie, although if the student enrolls in the free trial,
they are tracked by user-id from that point forward. The same user-id cannot
enroll in the free trial twice. For users that do not enroll, their user-id is not tracked
in the experiment, even if they were signed in when they visited the course
overview page.

## 2. Experiment Design<a class="anchor" id="design"></a>



## 2.1 Metric Choice<a class="anchor" id="choice"></a>

### Two types of metrics measure how the experiment group performs better than the control group - __Invariant Metrics__ and __Evaluation Metrics__. 

__[Invariant Metrics]__ are what we usually use for sanity checks - checking whether the distribution in control and experiment group is similar or not - It performing some consistent checking across all of our experiment. In another word, this means we would not expect a change between the experiment and the control group for these metrics.

   - __Number of cookies__: The number of unique cookies to view the course overview page ($d_{min}$=3000). This is the unit of diversion. People won't be able to see the prompt unless they click the "Start Free Trail" button, so the number of cookies is not expected to change. It is therefore appropriate as an invariant metric.

   - __Number of clicks__: The number of unique cookies to click the "Start free trial" button (which happens before the free trial screener is trigger) ($d_{min}$=240). The same reason of counting the "Number of cookies" as an invariant can be applied here - we wouldn't expect to see a change between the experiment and control group since at this point, student who click the "Start free trail" button haven't get a chance to see the prompt of time commitment.     
   - __Click-through-probability__: That is, number of unique cookies to click the "Start free trial" button divided by number of unique cookies to view the course overview page ($d_{min}$=0.01). Since the number of cookie and number of clicks are both invariant due to the fact that the user behavior won't change before they see the new feature we tested on, hence the Click-through-probability is a good invariant metric.



__[Evaluation Metrics]__ are the metrics in which we expect to see a difference or change between the experiment and the control group as a function of the experiment. Aiming at the minimizing the student frustration and maximizing the limited coaching resource as the ultimate business goal, each evaluation would have a minimum difference __($d_{min}$)__ that would have to observed before that was a meaningful change for the business.


   - __Gross conversion__: That is, number of user-ids to complete checkout and enroll in the free trial divided by number of unique cookies to click the "Start free trial" button ($d_{min}$= 0.01). We are expecting to see a reduced gross conversion in the experiment group.
   
   - __Retention__: That is, number of user-ids to remain enrolled past the 14-day boundary (and thus make at least one payment) divided by number of user-ids to complete checkout ($d_{min}$=0.01).
   
   - __Net conversion__: That is, number of user-ids to remain enrolled past the 14-day boundary (and thus make at least one payment) divided by the number of unique cookies to click the "Start free trial" button ($d_{min}$= 0.0075).
    
  
### *Launch Criteria*

In order to launch this prompt, the following condition should be considered - 

  
 - __Decreases Gross Conversion coupled to Increased Net Conversion__. 
 We would expect to see 
    1. less student enrolling in the free trail 
    2. but more student staying beyond the free trail. More specifically, Gross conversion is both statistically and practically significant lower, and net conversion is not lower than the practical limit ( -0.75%).  

We will launch the new feature if both criteria are fulfilled.    



__[Unused Metrics]__

   - __Number of user-ids__: The number of users who enroll in the free trial ($d_{min}$=50). This is neither an invariant metric nor an evaluation metric. User-ids are tracked only after enrolling in the free trail so we would not expect equal distribution between the control and experience group. Besides, this metric is not robust enough to evaluate the experiment. A row count might be able to track the students stayed beyond the 14 day free trail boundary, however, it's hard to evaluate the change or difference in the same scale - that's also why Gross Conversion is a good evaluation metric, it is normalized by the number of cookies. 

## 2.2 Measuring Variability<a class="anchor" id="variability"></a>

In this section, we will be using the baseline value to estimate the standard deviation for the evaluation metrics analytically, in which we can get to know the range of the metrics will be used also how spread our metrics are. To actually to calculate the standard deviation given the baseline value - assuming we are estimate that for 5,000 cookies visiting the course. 

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

from IPython.display import display


In [2]:
# load the baseline values data
baseline = pd.read_csv("/Users/jinghongjiangyang/Desktop/ABT/Final_Project/data/Baseline_Values.csv", header =None)

# rename the data frame columns
baseline.columns = ['Metric', 'Value']

# view the data
print("-"*10, "Original Baseline Value ", "-"*10)
display(baseline.head(7))


# since we only estimate the SD for evaluation metrics in 5,000 sample
# we will rescale the base line value as what we need
# make a copy of the original data
baseline_rescale = baseline.copy()
# rescale the counts value to the size of 5,000
baseline_rescale.iloc[[0,1,2], 1] = baseline_rescale.iloc[[0,1,2], 1] * (5000/40000)
# preview the scaled data
print("-"*10, "Rescaled Baseline Value", "-"*10)
baseline_rescale.head(7)


---------- Original Baseline Value  ----------


,Metric,Value
0,Unique cookies to view overview page per day,40000.000000
1,"Unique cookies to click ""Start free trial"" per day",3200.000000
2,Enrollments per day,660.000000
3,"Click-through-probability on ""Start free trial""",0.080000
4,"Probability of enrolling, given click",0.206250
5,"Probability of payment, given enroll",0.530000
6,"Probability of payment, given click",0.109313


---------- Rescaled Baseline Value ----------


,Metric,Value
0,Unique cookies to view overview page per day,5000.000000
1,"Unique cookies to click ""Start free trial"" per day",400.000000
2,Enrollments per day,82.500000
3,"Click-through-probability on ""Start free trial""",0.080000
4,"Probability of enrolling, given click",0.206250
5,"Probability of payment, given enroll",0.530000
6,"Probability of payment, given click",0.109313


In order to access the standard deviation analytically, we assume the three evaluation metrics (probability) follow the binomial distribution and the formula for the standard distribution is:<br>
<center><font size="4">$SD=\sqrt{\frac{\hat{p}*(1-\hat{p})}{n}}$</font></center><br>

For each metric, the variables from formula refer to: <br>
 - $\hat{p}$ - baseline probability of the event <br>
 - $ n $ - sample size <br>

Note that in the case the __unit of diversion__ of the experiment is equal to the __unit of analysis__, the analytical estimate of standard deviation tends to be close to the empirical standard deviation. This is valid for Gross Conversion and Net Conversion but not Retention. if the retention will be the final evaluation metric to look at, we will need to calculate the empirical variability.

In [3]:
import numpy as np
# calculate the variance for Gross Conversion, Net Conversion and Retention

# a list of metric name
Metric = ['Gross Conversion', 'Retention', 'Net Conversion']
# a list of the sample size 
N = [400, 82.5 ,400]
# a list of the probability 
P = [0.20625, 0.53, 0.109313]

# plug in the number to get the SD for each metrics
for i, item in enumerate(Metric):
    SD = np.sqrt(P[i]*(1-P[i])/N[i])
    print(Metric[i], ": SD =",round(SD,4))

Gross Conversion : SD = 0.0202
Retention : SD = 0.0549
Net Conversion : SD = 0.0156


## 2.3 Sizing<a class="anchor" id="sizing"></a> 

## 2.3.1 Choosing Number of Samples Given Power<a class="anchor" id="samplesize"></a><br>

Pageviews required for each evaluation metric is calculated by the [online calculator](https://www.evanmiller.org/ab-testing/sample-size.html). We will be using $\alpha$ = 0.05 and $\beta$ = 0.2 to do the following calculation of the pageviews, which target at enough statistical power for each metric. Note that Bonferroni correction won't be applied here - we only have three/two evaluation metrics to look at and these three metrics are not independent to each other - it will make the estimation too conservative.

__Gross Conversion__

- Baseline Conversion: 20.625%
- Minimum Detectable Effect: 1%
- Alpha: 5%
- Beta: 20% 
- Sensitivity (1 - Beta): 80%
- Sample Size = 25,835 enrollments/group
- Number of groups = 2 (experiment and control)
- Total sample size = 51,670 enrollments
- Clicks/Pageview: 3200/40000 = 0.08 clicks/pageview
- Pageviews Required = 645,875

__Retention__

- Baseline Conversion: 53%
- Minimum Detectable Effect: 1%
- Alpha: 5%
- Beta: 20% 
- Sensitivity (1 - Beta): 80%
- Sample Size = 39,115 enrollments/group
- Number of groups = 2 (experiment and control)
- Total sample size = 78,230 enrollments
- Enrollments/Pageview: 82.5/5000 = 0.0165 enrollments/pageview
- Pageviews Required = 4,741,212

__Net Conversion__

- Baseline Conversion: 10.9313%
- Minimum Detectable Effect: 0.75%
- Alpha: 5%
- Beta: 20% 
- Sensitivity (1 - Beta): 80%
- Sample Size = 27,413 enrollments/group
- Number of groups = 2 (experiment and control)
- Total sample size = 54,826 enrollments
- Clicks/Pageview: 3200/40000 = 0.08 clicks/pageview
- Pageviews Required = 685,325

## 2.3.2 Choosing Duration vs. Exposure<a class="anchor" id="duration"></a>

Let's start with the maximum pageviews required to use based on the metrics retention - 4,741,212, if we divert 100% of our traffic, given 40,000 pageviews per day, the experiment will take about 117 days. However, if we stick to the Gross Conversion and Net Conversion, the sample size required will be reduced to 685,325 pageviews, and that'll take about 18 days with 100% diversion and 35 days with 50% diversion.

Given that this experiment is at low risk and harmless to the student, I recommend running this experiment with full traffic to get the results as soon as possible. The 18 days would be long enough to capture the student behavior after 14 days boundary - whether they cancel it or not. Also, the shorter exposure period will minimize the possible confusion from student asking each other whether one see the new feature or not.       


## 3. Experiment Analysis<a class="anchor" id="analysis"></a><br>

There are two group of data in the experiment will be used for analysis.
 -  [Control Group](https://docs.google.com/spreadsheets/d/1Mu5u9GrybDdska-ljPXyBjTpdZIUev_6i7t4LRDfXM8/edit#gid=0) 
 -  [Experiment Group](https://docs.google.com/spreadsheets/d/1Mu5u9GrybDdska-ljPXyBjTpdZIUev_6i7t4LRDfXM8/edit#gid=154400404) 

## 3.1 Sanity Checks<a class="anchor" id="checks"></a>

Before rushing into the results interpretation, we need to do sanity check for the invariant metrics, the invariant metrics are not supposed to change before and after the experiment - and by confirming that from out data, can we make sure the comparable results for both experiment and control group.

To check the invariant metrics, we will follow the steps below,
 - 1. Create summary of metric for both control and experimental group.
 - 2. Compute standard deviation with probability 0.5 - since the chance of in control/experiment group should be equal.
 - 3. Multiple Z-score to get the Margin of Error.
 - 4. Check whether the observed fraction is within the interval.


In [4]:
# load the results data
# control group & experiment group
control = pd.read_csv("/Users/jinghongjiangyang/Desktop/ABT/Final_Project/data/Control.csv")
experiment = pd.read_csv("/Users/jinghongjiangyang/Desktop/ABT/Final_Project/data/Experiment.csv")

# preview the data
print("-"*13, "Control Group", "-"*13)
display(control.head())
display(control.tail())

print("-"*13, "Experimental Group ", "-"*13)
display(experiment.head())
display(experiment.tail())

------------- Control Group -------------


,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7723,687,134.0,70.0
1,"Sun, Oct 12",9102,779,147.0,70.0
2,"Mon, Oct 13",10511,909,167.0,95.0
3,"Tue, Oct 14",9871,836,156.0,105.0
4,"Wed, Oct 15",10014,837,163.0,64.0


,Date,Pageviews,Clicks,Enrollments,Payments
32,"Wed, Nov 12",10134,801,NaN,NaN
33,"Thu, Nov 13",9717,814,NaN,NaN
34,"Fri, Nov 14",9192,735,NaN,NaN
35,"Sat, Nov 15",8630,743,NaN,NaN
36,"Sun, Nov 16",8970,722,NaN,NaN


------------- Experimental Group  -------------


,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7716,686,105.0,34.0
1,"Sun, Oct 12",9288,785,116.0,91.0
2,"Mon, Oct 13",10480,884,145.0,79.0
3,"Tue, Oct 14",9867,827,138.0,92.0
4,"Wed, Oct 15",9793,832,140.0,94.0


,Date,Pageviews,Clicks,Enrollments,Payments
32,"Wed, Nov 12",10042,802,NaN,NaN
33,"Thu, Nov 13",9721,829,NaN,NaN
34,"Fri, Nov 14",9304,770,NaN,NaN
35,"Sat, Nov 15",8668,724,NaN,NaN
36,"Sun, Nov 16",8988,710,NaN,NaN


In [5]:
# define the function for sanity check ------------------------------------------------;

# for metrics that are countd
def sanity_check_cnt(metric, n_ctl, n_exp, Z):
    p = 0.5
    sd = np.sqrt(p*(1-p)/(n_ctl+n_exp))
    moe = sd*Z 
    CI_lower = round(p-moe, 4)
    CI_upper = round(p+moe, 4)
    p_hat = round(n_ctl/(n_ctl+n_exp), 4)

    if p_hat >= CI_lower and p_hat<CI_upper:
        print(metric,":", p_hat, ", 95% CI: {}, ".format([CI_lower, CI_upper]), "Sanity Check: Pass")
    else:
        print(metric," :", p_hat, ", 95% CI: {}, ".format([CI_lower, CI_upper]), "Sanity Check: Not Pass")
        
        
# for metrics that are probability        
def sanity_check_prb(metric, num_ctl, num_exp, denom_ctl, denom_exp, Z):
    p_ctl = num_ctl/denom_ctl
    p_exp = num_exp/denom_exp
    d_hat = round(p_exp - p_ctl, 4)
    p_pooled = round((num_ctl+num_exp)/(denom_ctl+denom_exp), 4)
    sd_pooled = round(np.sqrt(p_pooled*(1-p_pooled)*(1/denom_ctl + 1/denom_exp)),4)
    moe = sd_pooled*Z
    CI_lower = round(0-moe, 4)
    CI_upper = round(0+moe, 4)
    
    if d_hat >= CI_lower and d_hat<CI_upper:
        print(metric,":", d_hat, ", 95% CI: {}, ".format([CI_lower, CI_upper]), "Sanity Check: Pass")
    else:
        print(metric," :", d_hat, ", 95% CI: {}, ".format([CI_lower, CI_upper]), "Sanity Check: Not Pass")
        

# get the data from the results -------------------------------------------------------;
pageviews_ctl = control['Pageviews'].sum()
pageviews_exp = experiment['Pageviews'].sum()

clicks_ctl = control['Clicks'].sum()
clicks_exp = experiment['Clicks'].sum()

# Apply the sanity check function to our data -----------------------------------------;

# sanity check for Number of cookies
sanity_check_cnt("Number of cookies (P_ctl)", pageviews_ctl, pageviews_exp, 1.96 )

# sanity check for Number of clicks
sanity_check_cnt("Number of clicks (P_ctl)", clicks_ctl, clicks_exp, 1.96 )

# sanity check for Click-through-probability: 
sanity_check_prb("Click-through-probability (diff.)", clicks_ctl, clicks_exp, pageviews_ctl, pageviews_exp,  1.96  )
       

Number of cookies (P_ctl) : 0.5006 , 95% CI: [0.4988, 0.5012],  Sanity Check: Pass
Number of clicks (P_ctl) : 0.5005 , 95% CI: [0.4959, 0.5041],  Sanity Check: Pass
Click-through-probability (diff.) : 0.0001 , 95% CI: [-0.0014, 0.0014],  Sanity Check: Pass


## 3.2 Result Analysis<a class="anchor" id="resultanalysis"></a> 

## 3.2.1 Effect Size Tests<a class="anchor" id="effectsize"></a> 

For each evaluation metrics, given 95% confidence interval around the difference between the experiment and control group. Let's take a look at whether each evaluation metric is statistically and practically significant.

In [6]:
# define the function for metrics calculation ----------------------------------------------;
def exam_effect_size(metric, dmin, num_ctl, num_exp, denom_ctl, denom_exp, Z):
    p_ctl = num_ctl/denom_ctl
    p_exp = num_exp/denom_exp
    diff_obs = round(p_exp - p_ctl, 4)
    p_pooled = round((num_ctl+num_exp)/(denom_ctl+denom_exp), 4)
    sd_pooled = round(np.sqrt(p_pooled*(1-p_pooled)*(1/denom_ctl + 1/denom_exp)),4)
    moe = sd_pooled*Z
    CI_lower = round(diff_obs-moe, 4)
    CI_upper = round(diff_obs+moe, 4)
    
    print(metric, ': dmin = {}, d_obs = {},'.format(dmin, diff_obs), "95% CI {},".format([CI_lower, CI_upper]))
    if 0 > CI_upper or 0 < CI_lower:
        print("Statistical Significant: Yes")
    else:
        print("Statistical Significant: No")
        
    if np.sign(diff_obs)*dmin >= CI_upper or np.sign(diff_obs)*dmin  <= CI_lower:
        print("Practical Significant: Yes")
    else:
        print("Practical Significant: No")
        
    
        
# prepare the data to use ------------------------------------------------------------------;
clicks_ctl = control['Clicks'].loc[control['Enrollments'].notnull()].sum()
clicks_exp = experiment['Clicks'].loc[experiment['Enrollments'].notnull()].sum()

enrollments_ctl = control['Enrollments'].sum()
enrollments_exp = experiment['Enrollments'].sum()

payment_ctl = control['Payments'].sum() 
payment_exp = experiment['Payments'].sum()


### Gross Conversion

In [7]:
exam_effect_size("Gross Conversion", 0.01, enrollments_ctl, enrollments_exp, clicks_ctl, clicks_exp, 1.96 )

Gross Conversion : dmin = 0.01, d_obs = -0.0206, 95% CI [-0.0292, -0.012],
Statistical Significant: Yes
Practical Significant: Yes


### Net Conversion

In [8]:
exam_effect_size("Net Conversion", 0.0075, payment_ctl, payment_exp, clicks_ctl, clicks_exp, 1.96 )

Net Conversion : dmin = 0.0075, d_obs = -0.0049, 95% CI [-0.0116, 0.0018],
Statistical Significant: No
Practical Significant: No


The results for Gross Conversion is both statistically and practically significant, And the Net Conversion is neither statistically nor practically significant. Though one caveat has been noticed is that the 95% CI for net conversion is [-0.0116, 0.0018], which went down below the practical limit (-0.0075), which means there is risks of the decreased net conversion if the feature is launched.  

## 3.2.2 Sign Tests<a class="anchor" id="signtest"></a><br>
In the sign test we check our results from another perspective - if the trend change we observed was evident in the daily data as well. We will be examining the data from control and experiment group day by day. We can complete this by the [Online Tool](https://www.graphpad.com/quickcalcs/binomial1/) or do it manually.

In [9]:
import warnings
warnings.filterwarnings("ignore")

In [10]:
# Merge the two dataset by Date
full = pd.merge(control.iloc[:,[0,2,3,4]], experiment.iloc[:,[0,2,3,4]], 
                how='inner', on='Date', suffixes=('_ctl', '_exp'))

# Drop all the missing values
full_clean = full.dropna()

# create two flag variables to indicate the value change
full_clean['GC_flag'] = np.where(full_clean['Enrollments_ctl']/full_clean['Clicks_ctl'] < full_clean['Enrollments_exp']/full_clean['Clicks_exp'], 1, 0)
full_clean['NC_flag'] = np.where(full_clean['Payments_ctl']/full_clean['Clicks_ctl'] < full_clean['Payments_exp']/full_clean['Clicks_exp'], 1, 0)

# view the data
display(full_clean)

# print the number required for the sign test 
print("Total #. of Obs: {}".format(full_clean.shape[0]))
print("# of Negative cases for Gross Conversion: {}". format(full_clean.GC_flag[full_clean['GC_flag']==1].count()))
print("# of Negative cases for Net Conversion: {}". format(full_clean.GC_flag[full_clean['NC_flag']==1].count()))

,Date,Clicks_ctl,Enrollments_ctl,Payments_ctl,Clicks_exp,Enrollments_exp,Payments_exp,GC_flag,NC_flag
0,"Sat, Oct 11",687,134.0,70.0,686,105.0,34.0,0,0
1,"Sun, Oct 12",779,147.0,70.0,785,116.0,91.0,0,1
2,"Mon, Oct 13",909,167.0,95.0,884,145.0,79.0,0,0
3,"Tue, Oct 14",836,156.0,105.0,827,138.0,92.0,0,0
4,"Wed, Oct 15",837,163.0,64.0,832,140.0,94.0,0,1
5,"Thu, Oct 16",823,138.0,82.0,788,129.0,61.0,0,0
6,"Fri, Oct 17",748,146.0,76.0,780,127.0,44.0,0,0
7,"Sat, Oct 18",632,110.0,70.0,652,94.0,62.0,0,0
8,"Sun, Oct 19",691,131.0,60.0,697,120.0,77.0,0,1
9,"Mon, Oct 20",861,165.0,97.0,860,153.0,98.0,0,1


Total #. of Obs: 23
# of Negative cases for Gross Conversion: 4
# of Negative cases for Net Conversion: 10


## Sign and binomial test
### Growth Conversion
- Number of "successes": 4
- Number of trials (or subjects) per experiment: 23
- Sign test. If the probability of "success" in each trial or subject is 0.500, then:
- The two-tail P value is __0.0026*__
- This is the chance of observing either 4 or fewer successes, or 19 or more successes, in 23 trials.


### Net Conversion

- Number of "successes": 10
- Number of trials (or subjects) per experiment: 23
- Sign test. If the probability of "success" in each trial or subject is 0.500, then:
- The two-tail P value is __0.6776__
- This is the chance of observing either 10 or fewer successes, or 13 or more successes, in 23 trials.

Consistent with the effect size results, the Sign test results indicate that the change for Growth Conversion is statistical significant whereas that for Net Conversion is not.


## 3.2.3 Summary<a class="anchor" id="summary"></a> <br>

Based on the results of our experiment - I wouldn't recommend to launch the feature.

The results of the Gross Conversion is absolutely lower with the new feature, which indicates that adding this feature might be helpful to lower the frustration at the very first beginning. And our first criterion is fulfilled. However, there is some risk of declining profit (Net Conversion) since the confidence interval include the negative practical limit. The second condition is not passed. Recall that in the previous launch criteria section we predetermined that we only launch the experiment only if both condition are fulfilled.